In [7]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import pandas as pd
import mlflow

In [8]:
data = load_iris()
x = data['data']
y = data['target']

In [9]:
x[:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [10]:
y[:3]

array([0, 0, 0])

In [11]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.3,random_state=23,shuffle=True)

In [12]:

# Define models and their hyperparameter grids
models = [
    ("LogisticRegression", LogisticRegression(), {"C": 0.1, "solver": "liblinear"}),
    ("SupportVectorMachine", SVC(), {"C": 0.1, "kernel":  "rbf"}),
    ("DecisionTreeClassifier", DecisionTreeClassifier(), {"max_depth": 10, "criterion":"gini"})
]


In [13]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Mlflow_Test1")

<Experiment: artifact_location='mlflow-artifacts:/876050348167757470', creation_time=1734427913051, experiment_id='876050348167757470', last_update_time=1734427913051, lifecycle_stage='active', name='Mlflow_Test1', tags={}>

In [14]:
for name,model,params in models:
    model.set_params(**params)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    class_report = classification_report(y_test,y_pred,output_dict=True)

    accuracy = class_report['accuracy']
    precision = class_report['weighted avg']['precision']
    recall = class_report['weighted avg']['recall']
    support = class_report['weighted avg']['support']

    with mlflow.start_run(run_name=name):

        mlflow.log_params(params)
        mlflow.log_param('model_name',name)
        mlflow.log_metric('accuracy',accuracy)
        mlflow.log_metric('precision',precision)
        mlflow.log_metric('recall',recall)
        mlflow.log_metric('support',support)

        mlflow.sklearn.log_model(model,'model')

2024/12/17 15:54:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/876050348167757470/runs/3815538aaf5f456082c792c21e98e922
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/876050348167757470


2024/12/17 15:54:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SupportVectorMachine at: http://127.0.0.1:5000/#/experiments/876050348167757470/runs/abef4c0f0c004eb7b19bcdeeb7783f79
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/876050348167757470


2024/12/17 15:54:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run DecisionTreeClassifier at: http://127.0.0.1:5000/#/experiments/876050348167757470/runs/8e7d5024fe774d2b8b7ffb8176167a72
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/876050348167757470


In [15]:
result = mlflow.register_model('runs:/e1b066532f6549fb8966253377770f90/model','DecisionTreeClassifier')

Successfully registered model 'DecisionTreeClassifier'.
2024/12/17 15:54:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DecisionTreeClassifier, version 1
Created version '1' of model 'DecisionTreeClassifier'.


In [18]:
model = mlflow.sklearn.load_model('models:/DecisionTreeClassifier@challenger')
model

DecisionTreeClassifier(max_depth=10)